In [ ]:
%matplotlib auto



# Invoice Reading Code


In [ ]:
# Import required packages 
import cv2 
import pytesseract
import numpy as np


file=input("Location of invoice")
output_file=input('output location of excel file')
# Mention the installed location of Tesseract-OCR in your system 
pytesseract.pytesseract.tesseract_cmd = 'C:/Users/cr712/AppData/Local/Tesseract-OCR/tesseract.exe'

# Read image from which text needs to be extracted 
if file.endswith('.jpg'):
    img = cv2.imread(file) 

    # Preprocessing the image starts 

    # Convert the image to gray scale 
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 
    kernel = np.array([[0, -1, 0], 
                       [-1, 5,-1], 
                       [0, -1, 0]])

    # Sharpen image
    image_sharp = cv2.filter2D(gray, -1, kernel)

    # Performing OTSU threshold 
    ret, thresh1 = cv2.threshold(image_sharp, 0, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY_INV) 
    #ret, thresh1 = cv2.threshold(gray, 120, 255, cv2.THRESH_TOZERO_INV)

    # Specify structure shape and kernel size. 
    # Kernel size increases or decreases the area 
    # of the rectangle to be detected. 
    # A smaller value like (10, 10) will detect 
    # each word instead of a sentence. 
    rect_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (30,50)) 

    # Appplying dilation on the threshold image 
    dilation = cv2.dilate(thresh1, rect_kernel, iterations = 1) 

    # Finding contours 
    image, contours, hierarchy = cv2.findContours(dilation, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE) 

    # Creating a copy of image 
    im2 = img.copy() 


    # A text file is created and flushed 
    file = open("recognized.txt", "w+") 
    file.write("") 
    file.close() 

    # Looping through the identified contours 
    # Then rectangular part is cropped and passed on 
    # to pytesseract for extracting text from it 
    # Extracted text is then written into the text file 
    i=1
    for cnt in contours: 
        x, y, w, h = cv2.boundingRect(cnt) 

        # Drawing a rectangle on copied image 
        rect = cv2.rectangle(im2, (x, y), (x + w, y + h), (0, 255, 0), 2) 

        # Cropping the text block for giving input to OCR 
        cropped = im2[y:y + h, x:x + w] 

        # Open the file in append mode 
        file = open("recognized.txt", "a") 

        # Apply OCR on the cropped image 
        text = pytesseract.image_to_string(cropped) 
        print(i,'\n',text)
        i=i+1
        # Appending the text into file 
        file.write('\n') 
        file.write(text) 

        # Close the file 
        file.close 
else:
    if file.endswith('.pdf'):
        from textgetter.gettxt import pdf_txt_extract
            pdf_txt_extract(file, output_file, ocr_path, verbose=True)




# Extracting data

In [ ]:
import regex
import unicodedata as ud
import re
#detecting currency
    cur=regex.findall(r'\p{Sc}', data)
    currency=ud.name(cur[0])

#detecting adress
#Check Keyword if present 
key_check=re.compile(r'((?i)(billed to)\d+ ((?! \d+ ).)*(, \d{5}| \d{5}|-\d{2}|\b)|((?i)(CLient name)\d+ ((?! \d+ ).)*(, \d{5}| \d{5}|-\d{2}|\b)|((?i)|((?i)(Ship)\d+ ((?! \d+ ).)*(, \d{5}| \d{5}|-\d{2}|\b)|((?i)(payable)\d+ ((?! \d+ ).)*(, \d{5}| \d{5}|-\d{2}|\b)',re.I|re.MULTILINE)

address=key_check.findall(data)
address=address[0]

#detecting invoice no
number_regex=re.compile(r'(\w+)(?=(\s)*invoice)|(\w+)(?=(\s)*invoice(\s)*number)|(\w+)(?=(\s)*ref)|(?<=invoice)(\s)((\w)+(-)?(\d)+)',re.I|re.MULTILINE)
invoice_no=number_regex.findall(data)
invoice_no=invoice_no[-1]

Sub_total=re.compile(r"((?i)(Total|Subtotal|Tax|Discount|Sales tax|Sample tax|Shipping tax|Tax rate Credit used|Grand total)((?!\d+).)*(/d{1}|\b))",re.I|re.MULTILINE)
subtotal=Sub_total.findall(data)
subtotal=subtotal[-1]

total=re.compile(r"((?i)(Total|tax)((?!\d+).)*(/d{1}|\b))",re.I|re.MULTILINE)
subtotal=total.findall(data)
total=subtotal[-1]

# writing to excel file

In [ ]:
def writeToXL(output_file,Invoice_no,address,currency,subtotal,total):
    wb=xl.load_workbook(output_file)

    ws=wb.active
    #wb.get_sheet_names()
    #column 1 invoice no.
    #column 2 address
    #column 3 currency
    #column 4 Subtotal
    #column 5 Total
    a=1
    while True:
        if ws.cell(row=a,column=a).value:
            a=a+1
        else:
            break
    ws.cell(row=a,column=1).value = invoice_no
    ws.cell(row=a,column=2).value = address
    ws.cell(row=a,column=3).value = currency
    ws.cell(row=a,column=4).value = subtotal
    ws.cell(row=a,column=5).value = total
    wb.save('Invoice_data.xlsx')